In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)

from global_model import GlobalModel
from supported_modles import Supported_modles
import utils
import numpy as np

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [20]:
selected_model = Supported_modles.MLP_classifier

In [21]:
clients, test_x, test_y = utils.set_data(selected_model,True, False)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [22]:
f1_before = []
for client in clients:
    client.init_empty_model(0.01)
    # client.x, client.x_test, client.y ,client.y_test = client.split_data(0.2)
    print(client.y.mean())
    try:
        client.train_model()
    except ValueError:
        print(client.name)
        f1_before.append(0)
        continue
    client.f1 = client.test_model_f1(test_y,test_x)
    f1_before.append(client.f1)

    print('.')

0.6821176470588235
.
0.6791764705882353
.
0.6806764705882353
.
0.680725
.
0.6803935094928166
.


In [23]:
f1_before

[0.8355047633392325,
 0.8337307933301538,
 0.8343417581085742,
 0.8401638191990852,
 0.840678992487179]

In [28]:
global_model = GlobalModel(selected_model)

In [29]:

NUMBER_OF_CLIENTS = len(clients)
applicable_clients = []
start = True

print(f'Starting new round!')
f1s = np.empty((0,NUMBER_OF_CLIENTS), float)
data_weights = arr = np.empty((0,NUMBER_OF_CLIENTS), float)

for client in clients:
    if client.f1 != 0:
        if start:
            print(f'Initializing Global Model')
            global_model.model = client.model
            start = False

        applicable_clients.append(client) 
        f1s = np.append(f1s, client.f1)
        data_weights = np.append(data_weights, client.x.shape[0])
    
round_weights = (f1s/sum(f1s) + data_weights/sum(data_weights))/2
print(round_weights)

global_model.update_global_model(applicable_clients,round_weights,selected_model)

# for client in clients:
#     print(client.name)
#     f1_fede = global_model.f1_score(client.x_test,client.y_test)
#     f1_local = client.f1

#     print(f'{f1_fede}:{f1_local}')

#     if f1_fede > f1_local:
#         print("f1 fede was better")
#         client.f1 = f1_fede
#         # local was better set to local values
#         client.model = global_model.model
            
# print(f'++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

Starting new round!
Initializing Global Model
[0.19678913 0.19657715 0.19665016 0.21445536 0.1955282 ]


/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


In [30]:
f1_before

[0.8355047633392325,
 0.8337307933301538,
 0.8343417581085742,
 0.8401638191990852,
 0.840678992487179]

In [31]:
global_model.f1_score(test_x,test_y)

0.7902381399844283